This notebook is auto generated by bot - : <B> Plexy </B><br/> 
An Auto-ML Baseline notebook solution for Classification and Regression problems 

<br/> <B>  author@ : Kaustuv Kunal</B>

<br/>Learning type : Regression
 
### Contents 

1. <B>Import Libraries</B> 
<br/> 

2.  <B>Read & Explore Data</B>  
        2.1 Fetch Dataset    
        2.2 Dataset Extract   
        2.3 Dataset Summary   
        2.4 Dataset Shape
        2.5 Missing Features count 
        2.6 Segregate Features & Target  

3. <B>Data Visualisation</B>
        3.1 Target Distribution  
        3.2 Correlation-Matrix (For Numeric Features) 
        3.3 Category Distribution (For Categorical Features)
        3.4 Missing Values Distribution      
 
4. <B>Data Prepration & Cleaning</B>
		4.1 Dropping Additional Features
        4.1 Dropping Less Correlated Numeric Features
        4.2 Missing Values Treatment     
        4.3 Target Skewness Transform
        4.4 One-hot-encoding and PCA Flag Setup      
        4.5 Train Test Split 

5. <B>Data Modelling</B>    
        5.1 Base Models     
            5.1.1 Ridge Regression
            5.1.2 Lesso Regression
            5.1.3 ElasticNet   
            5.1.4 SVM
            5.1.5 KNeighbors    
        5.2 Bagging Models      
            5.2.1 Random Forest    
            5.2.2 Extra Tree       
        5.3 Boosting Models       
            5.3.1 XGBoost     
            5.3.2 LightGBM  
            5.3.3 AdaBoost  
        5.4 Model Leaderboard     
<br/> 
6. <B>Advance Modeling</B>     
        6.1 Voting Regressor (Average)
        6.2 Voting Regressor (Weighted Average)      
        6.3 Stacking Model 
        6.4 Final Model Leaderboard
         
7. <B>Submission File</B>

##  1. Import Libraries
Load the required libraries

In [ ]:
import warnings
warnings.simplefilter(action='ignore')
## modelling libraries

# read & explore Data
import pandas as pd
import numpy as np 
import itertools
import os 

# data visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# data  visualisation & prepration
from sklearn.model_selection import train_test_split


# data modeling
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from mlxtend.regressor import StackingCVRegressor
from sklearn.decomposition import TruncatedSVD

print (" .. libraries imported  ")

## 2.  Read & Explore Data

### 2.1 Fetch Data

In [ ]:
## fetch dataset

ext = "csv"
train_path = "/Users/kaustuv/DataScience/DS_projects/AnalyticVidhya/Reg_BlackFridaySale/datasets/train.csv"
test_path = "/Users/kaustuv/DataScience/DS_projects/AnalyticVidhya/Reg_BlackFridaySale/datasets/test.csv"


if( ext == "csv"):
	train_df = pd.read_csv(train_path)
elif (ext == "xlsx"):
	train_df = pd.read_excel(train_path)

train_copy = train_df.copy()


test_df = pd.DataFrame()
if os.path.exists(test_path):
	if( ext == "csv"):
		test_df = pd.read_csv(test_path)
	elif (ext == "xlsx"):
		test_df = pd.read_excel(test_path)
	

print (" ... dataset fetched")

### 2.2 Dataset Extract

In [ ]:
## dataset extract
train_df.head()

### 2.3 Dataset Summary

In [ ]:
## dataset  summary
train_df.describe()

In [ ]:
## find categorical columns in the dataset 
num_cols = train_df._get_numeric_data().columns
cat_cols = list(set(train_df.columns) - set(num_cols))

print ("There are " + str(len(num_cols)) + " numerical columns in the dataset (including one target column)")
print ("There are " + str(len(cat_cols)) + " object type columns in the dataset")

### 2.4 Dataset Shape

In [ ]:
# dataset shape
print (f'Shape of training data :{train_df.shape}')
print (f'Shape of testing data: {test_df.shape}')

### 2.5 Missing Features count

In [ ]:
nulls = pd.DataFrame(train_df.isnull().sum().sort_values(ascending=True), columns = ['Null_Counts'])
nulls_filtered = nulls[nulls['Null_Counts'] > 0]
if(nulls_filtered.shape[0] < 1):
    missing_flag = False
    print("There are no missing values in dataset ")
else:
    missing_flag = True
    missing_cols =  nulls_filtered.index
    numeric_missing_cols =  np.intersect1d(num_cols,missing_cols )
    categorical_missing_cols = np.intersect1d( cat_cols ,missing_cols )
    print (f' Total number of columns missing : { nulls_filtered.shape[0] }')
    print (f' Number of numerical columns missing : {len(numeric_missing_cols)} ')
    print (f' Number of objects columns missing : {len(categorical_missing_cols)} ')

### 2.6 Segregate Features & Target

In [ ]:
## segregate feature and target
target = "User_ID"
y = train_df[target]
distinct_Y = y.value_counts().index

##  we will remove id   from feature set

_id = "ID"

if _id == "": ## if id is not present, create a dummy 
    _id = 'id'
    train_df[_id] = 1
    test_df[_id] = 1
if _id not in list(test_df.columns):
    test_df[_id] = 1   

## drop the id columns : we will remove target at end 
test_id = test_df[_id]
train_df = train_df.drop([_id], axis=1)
test_df = test_df.drop([_id], axis=1)

## 3. Data Visualisation

### 3.1 Target Distribution

In [ ]:
train_df[target].hist(bins = 100, figsize = (10,10),density=True)

### 3.2  Correlation-Matrix  (For Numeric Features)

In [ ]:
# most correlated features
# todo -> have to use get_corr values later 

get_corr = False
corr = train_copy.corr()
if len(corr) > 0:
    get_corr = True
    top_corr_features = corr.index[abs(corr[target])>0.4]
    plt.figure(figsize=(10,10))
    g = sns.heatmap(train_copy[top_corr_features].corr(),annot=True,cmap="RdYlGn")

else:
    print ("No numeric variables for correlation plot")

### 3.3 Category Distribution (For Categorical Features)
Lets see how many category each categorical variable has

In [ ]:
# no of categories in categorical feature
col_list = train_df.select_dtypes(include='object').columns
cat_df = pd.DataFrame(columns= ["Feature","Category_Count"])
category_flag = False
if(len(col_list)!=0):
    category_flag= True
    cat_df = pd.DataFrame(columns= ["Feature","Category_Count"])
    for i in col_list: 
        n =  train_df[i].nunique()
        cat_df = cat_df.append({'Features': i, 'Category_Count': n }, ignore_index=True)
        cat_df.sort_values("Category_Count", axis = 0, ascending = True, inplace = True, na_position ='last')
    size = max(10, cat_df.shape[0])
    cat_df.plot(kind='barh',y='Category_Count', x ='Features', title = " No of Categories",color = ['g','y','b' ],stacked=True,legend= False,
                figsize=(size,size/2))
else:
    
    print ("No categorical variables in the dataset")

### 3.4 Missing Values Distribution

In [ ]:
if (missing_flag):
    row_count = train_df.shape[0]
    nulls_filtered['Null_percent'] = nulls_filtered['Null_Counts']/row_count
    nulls_filtered.plot(kind='barh',y='Null_percent',title = "Missing features percentage",
                    color = ['g','y','r' ],stacked=True,legend= False)
    print (f' Total No of missing columns: { nulls_filtered.shape[0] }')
    print (f' No of numerical columns missing: {len(numeric_missing_cols)} ')
    print (f' No of objects columns missing: {len(categorical_missing_cols)} ')
else:
    print("There are no missing values in dataset")

##  4. Data Prepration & Cleaning

### 4.1 Dropping Additional Features

In [ ]:
# dropping additional training set features (if any) wrt to test set
if (train_df.shape[1]-test_df.shape[1] >1):
    uncommon_feature = set(train_df.columns)- set(test_df.columns)
    uncommon_feature.remove(target)
    print(f'Dropping additional features: {uncommon_feature}')
    train_df.drop((list(uncommon_feature)), axis = 1 ,inplace = True)
else :
    print('No additional features to drop')

### 4.2 Dropping Less Correlated Numeric Features

Lets Remove less co-related numerical features

- threshhold_no_of_feature = 10 
- threshhold_corr_value = 0.4
- We will keep more than 'threshhold_no_of_feature' only if feature has correlation value larger than 
'threshhold_corr_value'
- Note: if you feel above value are not good enough for your dataset, reset this value and rerun the notebook

In [ ]:
#droping less co-related numerical features
if(get_corr):
    threshhold_no_of_feature = 10 
    threshhold_corr_value = 0.4
    corr_matrix =  corr.abs()    
    cm = pd.DataFrame(corr_matrix[target].sort_values(ascending=True) )
    cm['Features'] =cm.index
    cm = cm.reset_index()
    least_num_cor_feature = []
    for index, row in cm.iterrows():
        if index > threshhold_no_of_feature and row[target] < threshhold_corr_value :
            least_num_cor_feature.append(row["Features"])
    # drop the lesser corelated fetures from dataset 
    print (f'Shape of training data :{train_df.shape}')
    print (f'Shape of training data :{test_df.shape}')
    try:
        test_df =  test_df.drop((least_num_cor_feature), axis = 1)
        train_df = train_df.drop((least_num_cor_feature), axis = 1)
    except:
        pass
    print (f'Shape of training data after less correlated Numerical features removal  :{train_df.shape}')
    print (f'Shape of testing data after less correlated Numerical features removal :{test_df.shape}')
else:
    print ("No numeric variables for correlation")

### 4.2 Missing Values Treatment  

Rule for missing values treatment: 

For categorical variables: 
    - if missing percentage < 20% --> replace by mode
    - else -> add new category as missing

For continuous variables: 
    - if missing percentage > 20% --> remove feature column
    - else --> replace by mean

In [ ]:
if (missing_flag):
    #Remove features if missing percentage > 20
    numeric_missing_cols = np.array(list(set(numeric_missing_cols)- set(least_num_cor_feature) )) 
    
    high_missings_cols = (nulls_filtered[nulls_filtered['Null_percent'] > 0.2]).index   
    high_missings_numeric_cols = np.intersect1d(numeric_missing_cols,high_missings_cols )
    high_missings_categorical_cols = np.intersect1d(categorical_missing_cols,high_missings_cols )    
    lesser_missings_numeric_cols = np.setxor1d(numeric_missing_cols,high_missings_numeric_cols)   
    lesser_missings_categorical_cols= np.setxor1d(categorical_missing_cols,high_missings_categorical_cols )
    
    print ('Droping numeric features with missing percentage > 20%...')
    test_df =  test_df.drop((high_missings_numeric_cols), axis = 1)
    train_df = train_df.drop((high_missings_numeric_cols), axis = 1)
    print("Rest numerical columns, replacing the missing values by mean...")
    test_df[lesser_missings_numeric_cols] = test_df[lesser_missings_numeric_cols
                                                   ].fillna(test_df[lesser_missings_numeric_cols].mean())
    train_df[lesser_missings_numeric_cols] = train_df[lesser_missings_numeric_cols
                                                     ].fillna(train_df[lesser_missings_numeric_cols].mean())
    print ('Adding new category for categorical features with missing percentage > 20%...')
    train_df[high_missings_categorical_cols] = train_df[high_missings_categorical_cols].fillna('MISSING')
    test_df[high_missings_categorical_cols] =  test_df[high_missings_categorical_cols].fillna('MISSING')
    print ('Rest missing categorical features replacing by mode...')
    test_df[lesser_missings_categorical_cols] = test_df[lesser_missings_categorical_cols].fillna(test_df[lesser_missings_categorical_cols].mode().iloc[0])
    train_df[lesser_missings_categorical_cols] = train_df[lesser_missings_categorical_cols].fillna(train_df[lesser_missings_categorical_cols].mode().iloc[0])
    print (f'Shape of training data after less correlated Numerical features removal  :{train_df.shape}')
    print (f'Shape of testing data after less correlated Numerical features removal :{test_df.shape}')
else:
    print("There are no missing values in dataset")

#### Any more missing in test set

In [ ]:
if (missing_flag):
    # We have removed  mssing data from train set if test set has any missing items we will do again for test 
    test_num_cols = test_df._get_numeric_data().columns
    test_cat_cols = list(set(test_df.columns) - set(num_cols))
    test_row_count = test_df.shape[0]
    
    test_nulls = pd.DataFrame(test_df.isnull().sum().sort_values(ascending=True), columns = ['Null_Counts'])
    test_nulls_filtered = test_nulls[test_nulls['Null_Counts'] > 0]
    test_missing_cols =  test_nulls_filtered.index
    
    test_numeric_missing_cols =  np.intersect1d(test_num_cols,test_missing_cols )
    test_categorical_missing_cols = np.intersect1d( test_cat_cols ,test_missing_cols )
    # numeric by mean, objects by mode  
    test_df[test_numeric_missing_cols] = test_df[test_numeric_missing_cols].fillna(test_df[test_numeric_missing_cols].mean())
    test_df[test_categorical_missing_cols]=test_df[test_categorical_missing_cols].fillna(test_df.mode().iloc[0])
else:
    print("Skipping .. as there are no missing value")

In [ ]:
print (f'Shape of training data after missing value treatment  :{train_df.shape}')
print (f'Shape of testing data after missing value treatment :{test_df.shape}')

### 4.3 Target Skewness Transform

Transform target to minimize the skewness

In [ ]:
target_skew = np.absolute(train_copy[target].skew()) 
target_log_skew =np.absolute( np.log(train_copy[target]).skew())
target_sqrt_skew =np.absolute( np.sqrt(train_copy[target]).skew())
target_square_skew = np.absolute((train_copy[target]**2).skew())
skew_transform = 'normal'

m = min(target_skew,target_log_skew,target_sqrt_skew,target_square_skew)

if m == target_log_skew :
    y = np.log(train_copy[target])
    skew_transform = 'log'
elif m == target_sqrt_skew :
    y =  np.sqrt(train_copy[target])
    skew_transform = 'sqrt'
                
elif m == target_square_skew :
    y = train_copy[target]**2
    skew_transform = 'square'
    
print ( " target Skewness feature transform is "+skew_transform)

### 4.4  Categorical  Flag setup for one-hot-encoding  and PCA

We will prefer one-hot encoding but if any feature has many categories then we will enable PCA  before one hot encoding

In [ ]:
cat_col_flag = False
pca_flag = False
if(category_flag): 
    threshhold_no_of_feature = 50 
    max_cat_count= cat_df.iloc[-1]['Category_Count']
    #also take care of condition when no object feature 
    if (max_cat_count > threshhold_no_of_feature):
        pca_flag = True
    # category column flag    
    cat_col =  len(train_df.select_dtypes(include=object).columns)
    if (cat_col >0):
        cat_col_flag = True
    else:
        cat_col_flag = False
else:   
    print ("Skipping .. as no categorical variables in the dataset")

print (f'One-hot-encdoing flag : {cat_col_flag}') 
print (f'PCA flag : {pca_flag}')

### 4.5 Train Test Split

In [ ]:
train_df = train_df.drop([target], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(train_df, y, test_size=0.20, random_state=2019)

## 5. Data Modelling

Lets create Data models.
First we will define few useful functions

##### Function to evalute model

In [ ]:
metric = "neg_mean_squared_error"
def model_evaluate(model, metric=metric ):
    measure = cross_val_score(model,train_df, y, cv=3, scoring=metric )
    measure_mean = measure.mean()
    print("3 Fold Cross-Validation " +metric+" measure is : " +str(measure_mean))
    return  measure_mean

### 5.1 Base Models

In [ ]:
if (cat_col_flag & pca_flag  ):
    base_pipe = Pipeline([
                ('onehot',OneHotEncoder(handle_unknown='ignore')),
                ('pca',TruncatedSVD()),
                ('sscalar',  StandardScaler(with_mean=False)),
              ])
elif (cat_col_flag):
    base_pipe = Pipeline([
                ('onehot',OneHotEncoder(handle_unknown='ignore')),
                ('sscalar',  StandardScaler(with_mean=False)),
              ])
else:
    base_pipe = Pipeline([
                ('sscalar',  StandardScaler(with_mean=False)),
              ])


print('base model is :')
base_pipe

In [ ]:
# Dataframe for model leaderboard
models = pd.DataFrame( columns = ['Model', 'Model_Definition', 'Score'])

#### 5.1.1 Linear Ridge Regression

##### Training

In [ ]:
model1 =  Ridge()

model1_pipe = Pipeline([
                 ('base' , base_pipe),
                ('Ridge', model1),
              ])

model1_pipe.fit(X_train, y_train)
y_pred = model1_pipe.predict(X_valid)
model1_pipe_measure = model_evaluate(model1_pipe)

##### Hyperparameter tuning using RandomSearchCV

In [ ]:
params = {   
        "Ridge__alpha" : [ 1, 10, 100 ],
           }
model1_pipe_rs = RandomizedSearchCV( model1_pipe, params, random_state=21, cv=3, verbose=0, n_jobs=-1, scoring=metric)
best_model1_pipe = model1_pipe_rs.fit(X_train, y_train)


model1_def = best_model1_pipe.best_estimator_

model1_measure =  model_evaluate(model1_def)

models = models.append({'Model': 'Ridge', 'Model_Definition': model1_def, 'Score': model1_measure},
                       ignore_index=True)

print('\n'+"Model parameters after parameter tuning")
best_model1_pipe.best_params_

#### 5.1.2 Linear Lasso Regression

##### Training

In [ ]:
model2 =  Lasso()

model2_pipe = Pipeline([
                 ('base' , base_pipe),
                ('Lasso', model2),
              ])

model2_pipe.fit(X_train, y_train)
y_pred = model2_pipe.predict(X_valid)
model2_pipe_measure = model_evaluate(model2_pipe)

##### Hyperparameter tuning using RandomSearchCV

In [ ]:
params = {
        "Lasso__alpha" : [ 0.001, 0.01, 0.1,1, 10, 100 ],
           }
model2_pipe_rs = RandomizedSearchCV( model2_pipe, params, random_state=21, cv=3, verbose=0, n_jobs=-1, scoring=metric)
best_model2_pipe = model2_pipe_rs.fit(X_train, y_train)


model2_def = best_model2_pipe.best_estimator_

model2_measure =  model_evaluate(model2_def)

models = models.append({'Model': 'Lasso', 'Model_Definition': model2_def, 'Score': model2_measure},
                       ignore_index=True)
print('\n'+"Model parameters after parameter tuning")
best_model2_pipe.best_params_

#### 5.1.3 ElasticNet

##### Training

In [ ]:
model3 =  ElasticNet()

model3_pipe = Pipeline([
                 ('base' , base_pipe),
                ('ElasticNet', model3),
              ])

model3_pipe.fit(X_train, y_train)
y_pred = model3_pipe.predict(X_valid)
model3_pipe_measure = model_evaluate(model3_pipe)

##### Hyperparameter tuning using RandomSearchCV

In [ ]:
params = {
        "ElasticNet__alpha" : [ 0.001, 0.01, 0.1,1, 10, 100 ],
           }
model3_pipe_rs = RandomizedSearchCV( model3_pipe, params, random_state=21, cv=3, verbose=0, n_jobs=-1, scoring=metric)
best_model3_pipe = model3_pipe_rs.fit(X_train, y_train)

model3_def = best_model3_pipe.best_estimator_

model3_measure =  model_evaluate(model3_def)

models = models.append({'Model': 'ElasticNet', 'Model_Definition': model3_def, 'Score': model3_measure},
                       ignore_index=True)
print('\n'+"Model parameters after parameter tuning")
best_model3_pipe.best_params_

#### 5.1.4  SVM Regressor
##### Training

In [ ]:
model4 = SVR()
model4_pipe = Pipeline([
                 ('base' , base_pipe),
                ('SVR', model4),
              ])

model4_pipe.fit(X_train, y_train)
y_pred = model4_pipe.predict(X_valid)
model4_measure = model_evaluate(model4_pipe)

##### Hyperparameter tuning using RandomSearchCV

In [ ]:
params = {
        "SVR__C": [  0.001, 0.01, 0.1, 1, 10 ],
        "SVR__gamma" : [ 0.001, 0.01, 0.1, 1],

           }

model4_pipe_rs = RandomizedSearchCV( model4_pipe, params, random_state=21, cv=3, verbose=0, n_jobs=-1, scoring=metric)
best_model4_pipe = model4_pipe_rs.fit(X_train, y_train)


model4_def = best_model4_pipe.best_estimator_
model4_measure =  model_evaluate(model4_def)

models = models.append({'Model': 'SVR', 'Model_Definition': model4_def, 'Score': model4_measure},
                       ignore_index=True)
print('\n'+"Model parameters after parameter tuning")
best_model4_pipe.best_params_

#### 5.1.5  KNeighborsRegressor

##### Training

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

model5 = KNeighborsRegressor()
model5_pipe = Pipeline([
                 ('base' , base_pipe),
                ('KNeighborsRegressor', model5),
              ])

model5_pipe.fit(X_train, y_train)
y_pred = model5_pipe.predict(X_valid)
model5_measure = model_evaluate(model5_pipe)

##### Hyperparameter tuning using RandomSearchCV

In [ ]:
params = {
        "KNeighborsRegressor__n_neighbors": [  3, 5, 10 ],
        "KNeighborsRegressor__algorithm" : [ 'auto' ],
         }

model5_pipe_rs = RandomizedSearchCV( model5_pipe, params, random_state=21, cv=3, verbose=0, n_jobs=-1, scoring=metric)
best_model5_pipe = model5_pipe_rs.fit(X_train, y_train)


model5_def = best_model5_pipe.best_estimator_

model5_measure =  model_evaluate(model5_def)

models = models.append({'Model': 'KNeighborsRegressor', 'Model_Definition': model5_def, 'Score': model5_measure},
                       ignore_index=True)
print('\n'+"Model parameters after parameter tuning")
best_model5_pipe.best_params_

#### 5.1.6  Base Models leaderboard
We will take two top performing base model for our voting and stacking  model

In [ ]:
base_models_df = models.sort_values(by='Score', ascending=False)
base_models_df = base_models_df.set_index('Score')
# selecting top two base model
base_model1 = base_models_df.iloc[0]['Model_Definition']
base_model2 = base_models_df.iloc[1]['Model_Definition']

base_models_df.head(5)

### 5.2 Bagging Models

#### 5.2.1 RandomForestRegressor

##### Training

In [ ]:
model6 = RandomForestRegressor()
model6_pipe = Pipeline([
                 ('base' , base_pipe),
                ('RandomForestRegressor', model6),
              ])

model6_pipe.fit(X_train, y_train)
y_pred = model6_pipe.predict(X_valid)
mmodel6_pipe_measure = model_evaluate(model6_pipe)

##### Hyperparameter tuning using RandomSearchCV

In [ ]:
params = {
        'RandomForestRegressor__n_estimators': range(50,450,150),


           }
model6_pipe_rs = RandomizedSearchCV( model6_pipe, params, random_state=21, cv=3, verbose=0, n_jobs=-1, scoring=metric)
best_model6_pipe = model6_pipe_rs.fit(X_train, y_train)


model6_def = best_model6_pipe.best_estimator_

model6_measure =  model_evaluate(model6_def)

models = models.append({'Model': 'RandomForestRegressor', 'Model_Definition': model6_def, 'Score': model6_measure},
                       ignore_index=True)
print('\n'+"Model parameters after parameter tuning")
best_model6_pipe.best_params_

#### 5.2.2 ExtraTreesRegressor

##### Training

In [ ]:
model7 = ExtraTreesRegressor()
model7_pipe = Pipeline([
                 ('base' , base_pipe),
                ('ExtraTreesRegressor', model7),
              ])

model7_pipe.fit(X_train, y_train)
y_pred = model7_pipe.predict(X_valid)
model7_measure = model_evaluate(model7_pipe)

##### Hyperparameter tuning using RandomSearchCV

In [ ]:
params = {
        'ExtraTreesRegressor__n_estimators': range(50,126,25),
        'ExtraTreesRegressor__max_features': ['auto', None ],
        'ExtraTreesRegressor__min_samples_leaf': range(20,50,10),
        'ExtraTreesRegressor__min_samples_split': range(15,30,15),

           }

model7_pipe_rs = RandomizedSearchCV( model7_pipe, params, random_state=21, cv=3, verbose=0, n_jobs=-1, scoring=metric)
best_model7_pipe = model7_pipe_rs.fit(X_train, y_train)


model7_def = best_model7_pipe.best_estimator_

model7_measure =  model_evaluate(model7_def)

models = models.append({'Model': 'ExtraTreesRegressor', 'Model_Definition': model7_def, 'Score': model7_measure},
                       ignore_index=True)
print('\n'+"Model parameters after parameter tuning")
best_model7_pipe.best_params_

#### Select top bagging model for our stacking & voting regressor

In [ ]:
if (model6_measure > model7_measure):
    bagging_model = model6_def
else :
    bagging_model = model7_def

### 5.3 Boosting Ensemble Models

#### 5.3.1 XGBRegressor

##### Training

In [ ]:
model8 = XGBRegressor( )
model8_pipe = Pipeline([
                 ('base' , base_pipe),
                ('XGBRegressor', model8),
              ])

model8_pipe.fit(X_train, y_train)
y_pred = model8_pipe.predict(X_valid)
model8_measure = model_evaluate(model8_pipe)

##### Hyperparameter tuning using RandomSearchCV

In [ ]:
params = {
        "XGBRegressor__learning_rate": [  0.1, 0.01 ],
        "XGBRegressor__n_estimators" : [ 10, 100, 600],
        "XGBRegressor__objective" : [ 'reg:squarederror','reg:gamma'],
          "XGBRegressor__max_depth" : [ 4, 10 , 15 ],

           }

model8_pipe_rs = RandomizedSearchCV( model8_pipe, params, random_state=21, cv=3, verbose=0, n_jobs=-1, scoring=metric)

best_model8_pipe = model8_pipe_rs.fit(X_train, y_train)
model8_def = best_model8_pipe.best_estimator_

model8_measure =  model_evaluate(model8_def)
models = models.append({'Model': 'XGBRegressor', 'Model_Definition': model8_def, 'Score': model8_measure},
                       ignore_index=True)
print('\n'+"Model parameters after parameter tuning")
best_model8_pipe.best_params_

#### 5.3.2 LGBMRegressor

##### Training

In [ ]:
model9 = LGBMRegressor( )
model9_pipe = Pipeline([
                 ('base' , base_pipe),
                ('LGBMRegressor', model9),
              ])

model9_pipe.fit(X_train, y_train)
y_pred = model9_pipe.predict(X_valid)
model9_measure = model_evaluate(model9_pipe)

##### Hyperparameter tuning using RandomSearchCV

In [ ]:
params = {
        "LGBMRegressor__learning_rate": [  0.01, 0.1  ],
         "LGBMRegressor__num_iterations" : [ 100, 500 ],
           }

model9_pipe_rs = RandomizedSearchCV( model9_pipe, params, random_state=21, cv=3, verbose=0, n_jobs=-1, scoring=metric)

best_model9_pipe = model9_pipe_rs.fit(X_train, y_train)
model9_def = best_model9_pipe.best_estimator_

model9_measure =  model_evaluate(model9_def)

models = models.append({'Model': 'LGBMRegressor', 'Model_Definition': model9_def, 'Score': model9_measure},
                       ignore_index=True)
print('\n'+"Model parameters after parameter tuning")
best_model9_pipe.best_params_

#### 5.3.3 AdaBoostRegressor

##### Training

In [ ]:
model10 =  AdaBoostRegressor()
model10_pipe = Pipeline([
                 ('base' , base_pipe),
                ('AdaBoostRegressor', model10),
              ])

model10_pipe.fit(X_train, y_train)
y_pred = model10_pipe.predict(X_valid)
model10_measure = model_evaluate(model10_pipe)

##### Hyperparameter tuning using RandomSearchCV

In [ ]:
params = {
        "AdaBoostRegressor__learning_rate": [  0.01, 0.1, 1  ],
        "AdaBoostRegressor__n_estimators" : [ 100, 600, 1000],
           }

model10_pipe_rs = RandomizedSearchCV( model10_pipe, params, random_state=21, cv=3, verbose=0, n_jobs=-1, scoring=metric)
best_model10_pipe = model10_pipe_rs.fit(X_train, y_train)


model10_def = best_model10_pipe.best_estimator_

model10_measure =  model_evaluate(model10_def)

models = models.append({'Model': 'AdaBoostRegressor', 'Model_Definition': model10_def, 'Score': model10_measure},
                       ignore_index=True)
print('\n'+"Model parameters after parameter tuning")
best_model10_pipe.best_params_

#### Selecting top bosting model for our stacking & voting regressor

In [ ]:
if ( max(model10_measure,model9_measure,model8_measure) == model8_measure):
    boosting_model = model8_def
elif( max(model10_measure,model9_measure,model8_measure) == model9_measure):
    boosting_model = model9_def
else :
    boosting_model = model10_def

### 5.4 Model Leaderboard

In [ ]:
models_df = models.sort_values(by='Score', ascending=False)
models_df = models_df.set_index('Score')
models_df.head(10)

## 6. Advance Modeling
For our Advance Modeling, we will take 4 models two from base and one each from bagging and boosting model

### 6.1 Voting Regressor (Averaging)

Averages the individual predictions to form a final prediction.

In [ ]:
model11 = VotingRegressor([('base_model1',base_model1),('base_model2',base_model2), ('bagging_model', bagging_model),('boosting_model',boosting_model)] )
model11.fit(X_train,y_train)
y_pred = model11.predict(X_valid)
model11_measure = model_evaluate(model11)
# adding into our model list
models = models.append([{ 'Model': "Voting Regressor(Averaging)", 'Model_Definition': model11, 'Score': model11_measure }])

### 6.2 Voting Regressor (Weighted Averaging)

In [ ]:
model12 = VotingRegressor([('base_model1',base_model1),('base_model2',base_model2), ('bagging_model', bagging_model),('boosting_model',boosting_model)], weights= [0.15,0.15,0.15,0.55] )
model12.fit(X_train,y_train)
y_pred = model12.predict(X_valid)
model12_measure = model_evaluate(model12)
# adding into our model list
models = models.append([{ 'Model': "Voting Regressor (Weighted Averaging)", 'Model_Definition': model12, 'Score': model12_measure }])

### 6.2 Stacking Model

In [ ]:
lasso = Lasso()
model13 = StackingCVRegressor(regressors=(base_model1,bagging_model,boosting_model),
                            meta_regressor=lasso,
                            random_state= 2018)
X_train_matrix = X_train.as_matrix()
y_train_matrix = y_train.as_matrix()



model13.fit(X_train_matrix,y_train_matrix)
train_df_matrix = train_df.as_matrix()
mesure = cross_val_score(model13,train_df_matrix, y, cv=3, scoring= metric )

model13_measure = mesure.mean()
print("3 Fold Cross-Validation measure is : " +str(model13_measure))

models = models.append([{ 'Model': "StackingModel", 'Model_Definition': model13, 'Score': model13_measure }])

### 6.3 Final Model Leaderboard

In [ ]:
models_df = models.sort_values(by='Score', ascending=False)
models_df = models_df.set_index('Score')
models_df.head(13)

## 7. Submission File
Preparing submission file for the best model

In [ ]:
# function to perform unskewness
def consider_skewness(pred,sk_transform = skew_transform ):
    if sk_transform == 'log':
        return np.exp(pred)
    elif sk_transform == 'sqrt':
        return np.square(pred)
    elif sk_transform == 'square':
        return np.sqrt(pred)

    return pred

#### Generating Submission File

In [ ]:
# best model from all model
tuned_best_model_def = models_df.iloc[0]['Model_Definition']
# prediction
pred = tuned_best_model_def.predict(test_df)
# unskewness
final_pred = consider_skewness(pred)

sub = pd.DataFrame()
sub[_id] = test_id
sub[target] = final_pred
sub.to_csv("baseline_solution.csv", index=False)
print ("Submission File Generated, here is the snapshot: ")
print (sub.head(10))